# Executing Matlab code using Octave through python
## Libraries
For executing this mini-tutorial you need to have installed the following libraries:

In [1]:
! conda list octave

# packages in environment at /usr/local/share/miniconda3/envs/env:
#
# Name                    Version                   Build  Channel
octave                    7.3.0                h4bc5dab_3    conda-forge
octave_kernel             0.35.1             pyhd8ed1ab_0    conda-forge


In [2]:
! conda list oct2py

# packages in environment at /usr/local/share/miniconda3/envs/env:
#
# Name                    Version                   Build  Channel
oct2py                    5.6.1              pyhc1e730c_0    conda-forge


In [3]:
import subprocess
import oct2py as ocpy
from oct2py import octave as oc
import os

## Executing matlab/octave functions
Now you can execute matlab/octave as follows

### Option 1: OctPy()

Create an explicit instance of Oct2py given by the module oct2py. This allows more control within the Octave instance and how to interact with it.


In [6]:
# Instantiate octave executor
o2p= ocpy.Oct2Py()
# Run octave basic function
x = o2p.zeros(3, 3)
# Print the result
print(x)
# print result type
print(x.dtype)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
float64


### Option 2: octave
Uses a default Octave instanced given by oct2py. This allows to call simple Octave functions in a direct and simpler way without the explicit creation of a Oct2py instance. 

In [8]:
# Run octave basic function
x = ocpy.octave.zeros(3, 3)
# Print the result
print(x)
# print result type
print(x.dtype)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
float64


Each of the options has advantages and this advantages:

|                          | Predefined Instance (`oct2py.octave`) | Custom Instance (`oct2py.Oct2Py()`) |
|--------------------------|---------------------------------------|------------------------------------|
| **Advantages**           | - Simplicity                         | - Full control over the instance   |
|                          | - Less code                          | - Ability to have multiple         |
|                          |                                       |   instances                        |
|                          | - Quick usage                        | - Isolation of Octave environment  |
|                          |                                       | - Customization of parameters      |
| **Disadvantages**        | - Less control over the instance     | - More code and complexity         |
|                          |   of Octave                          |                                    |


In concret:

|                                    | Predefined Instance (`oct2py.octave`)                   | Custom Instance (`oct2py.Oct2Py()`)                      |
|------------------------------------|---------------------------------------------------------|----------------------------------------------------------|
| **Allows Configuration**           | No                                                      | Yes                                                      |
| **Allows Path Configuration**      | No                                                      | Yes                                                      |
| **Allows Environment Isolation**   | No (Shares environment with other code)                | Yes (Environment is isolated)                             |
| **Allows Multiple Instances**      | No (Single shared instance)                            | Yes (Multiple instances can be created)                  |
| **Allows Customization**           | Limited                                                 | Extensive                                                |
| **Allows Running Simple Functions**| Yes                                                     | Yes                                                      |
| **Allows Running Complex Scripts** | Yes                                                     | Yes                                                      |
| **Allows Passing Data**            | Yes                                                     | Yes                                                      |
| **Allows Returning Data**          | Yes                                                     | Yes                                                      |


... So ... let's see what happens if I want to execute a .m file

## Execute matlab/octave files from python
### Option 1: Oct2py()
This is the content in the file "hola_mundo.m" so we can check the result is the same

In [9]:
print(o2p.disp('Hola mundo'))




Hola mundo



If we do is as a donkey...

In [10]:
o2p.source('./octave-tests/hola_mundo.m')




Hola Mundo


But we are not donkeys! We are pro 😎

So... let's try to add the path to octave loads path

In [11]:
! ls octave-tests

hola_mundo.m


In [16]:
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"

print(path2add)
print(current_path)

o2p.addpath(path2add)  # doctest: +SKIP

if os.path.exists(path2add):
    o2p.addpath(path2add)
    print("Path correctly added to octave instance.")
else:
    print("The path is not available")


/home/macu/work/nbs_pipeline/octave-tests/
/home/macu/work/nbs_pipeline






Path correctly added to octave instance.


In [19]:
! ls {path2add}

hola_mundo.m


In [26]:
listing = o2p.dir(path2add)
myfile = ""
for file in listing:
    print(file.name);
    if file.name == "hola_mundo.m":
        myfile = file




.
..
.ipynb_checkpoints
hola_mundo.m


In [31]:
o2p.source(myfile.name)

Oct2PyError: Octave evaluation error:
error: run: function called with too many outputs
error: called from:
    run at line -1, column -1

Maybe not that pro...

Let's change matlab execution dir

In [33]:
print(path2add)
print(o2p.cd(path2add))


/home/macu/work/nbs_pipeline/octave-tests/



/home/macu/work/nbs_pipeline/octave-tests


In [35]:
o2p.source(myfile.name)




Hola Mundo


Now yes!! And still python and notebook in the correct place?

In [38]:
os.getcwd()
! dir

01_dataset_artifact.ipynb		   06-mplots-TESTS-toguether.ipynb
02a_encoder_DCAE.ipynb			   06-mplots-TESTS.ipynb
02b_encoder_MVP.ipynb			   __init__.py
02c_encoder_MVP-sliding_window_view.ipynb  __pycache__
03a_embeddings.ipynb			   _mplots_eammon_examples.ipynb
03b_embeddings-sliding_window_view.ipynb   artifacts
04a_dimensionality_reduction.ipynb	   config
04b_dimensionality_reduction-SWV.ipynb	   models
05-xai-lrp-TEST.ipynb			   mplot-explorer
05_xai-lrp.ipynb			   octave-tests
05_xai-lrp_.ipynb			   test_octave.ipynb
06-mplots-TESTS-SCAMP.ipynb		   test_torch.ipynb


Yes! So... We can setup the octove work path without modyfing python and console paths => We can have our matlab functions in a folder and call them without any hedeaches.

### Option 2: octave
Now let's see if ChatGPT lied to us when saying octove instance couldn't modify the load path.

In [45]:
print(ocpy.octave.disp('Hola mundo'))
ocpy.octave.source('./octave-tests/hola_mundo.m')




Hola mundo




Hola Mundo


In [46]:
# Setup the path
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"

print("add:", path2add)
print("current:", current_path)

print("--> Add path")
ocpy.octave.addpath(path2add)  # doctest: +SKIP


add: /home/macu/work/nbs_pipeline/octave-tests/
current: /home/macu/work/nbs_pipeline
--> Add path



--> Show final octave path


NameError: name 'path' is not defined

In [48]:
print("--> Show final octave path")
print(ocpy.octave.disp(ocpy.octave.path()))

--> Show final octave path






.:/home/macu/work/nbs_pipeline/octave-tests:/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/oct2py:/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/octave_kernel:/usr/local/share/miniconda3/envs/env/share/octave/site/m:/usr/local/share/miniconda3/envs/env/share/octave/site/m/startup:/usr/local/share/miniconda3/envs/env/lib/octave/7.3.0/oct/x86_64-conda-linux-gnu:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/audio:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/deprecated:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/elfun:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/general:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/geometry:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/gui:/usr/local/share/miniconda3/envs/env/share/octave/7.3.0/m/help:/usr/local/share/miniconda3/envs/env/share/octave/7.3.

In [49]:
print(ocpy.octave.source("hola_mundo.m"))

Oct2PyError: Octave evaluation error:
error: source: error sourcing file '/home/macu/work/nbs_pipeline/hola_mundo.m'

In [52]:
print(path2add)
print(ocpy.octave.cd(path2add))
ocpy.octave.source("hola_mundo.m")

/home/macu/work/nbs_pipeline/octave-tests/



/home/macu/work/nbs_pipeline/octave-tests



Hola Mundo


## Conclusions
ChatGPT lied to us! 
The load path can also be changued in the default 'octove' instance. 

Thus, the easier way to get the job done is: 

1) Import the libraries

In [1]:
import subprocess
import os
from oct2py import octave

2) Build and add your .m files path

In [2]:
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"
print(path2add)

if os.path.exists(path2add):
    octave.addpath(path2add)
    print("Path correctly added to octave instance.")
else:
    print("The path is not available")

/home/macu/work/nbs_pipeline/octave-tests/

Path correctly added to octave instance.


3) Move to the path and execute your .m file!

In [4]:
print(octave.cd(path2add))
octave.source("hola_mundo.m")




/home/macu/work/nbs_pipeline/octave-tests



Hola Mundo


4) And don't worry! The rest of the jupyter notebook will keep in its place

In [60]:
print("---> Python is here")
print(os.getcwd())
print("---> Console too!")
! pwd

---> Python is here
/home/macu/work/nbs_pipeline
---> Console too!
/home/macu/work/nbs_pipeline
